This notebook implements the Iterative Phase Estimation algorithm described in https://www.youtube.com/watch?v=aLSM0_H8hUE

In [2]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from pytket.circuit import CircBox, QControlBox
from pytket.extensions.qiskit import AerStateBackend
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus

In [3]:
"""
m: integer number of repetitions
omega: amount of Z rotation applied
state_prep_circuit: Circuit that prepares an eigenvector of unitary_circuit
unitary_circuit: circuit implementing a black box unitary
"""
def build_phase_estimation_circuit(
    m: int, omega: float, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", 1)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Add Hadamard gate to the measurement register
    qpe_circ.H(measurement_register[0])
    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
    # Run the controlled unitary m times
    for _ in range(m):
        qpe_circ.add_qcontrolbox(
            controlled_u_gate, list(measurement_register) + list(state_prep_register)
        )
    #alternate method of putting m iterations of U inside a single QControlBox
    
    # Um_circ = Circuit()
    # Um_state_register = Um_circ.add_q_register("v", unitary_circuit.n_qubits)
    # for _ in range(m):
    #     Um_circ.add_circuit(unitary_circuit, list(Um_state_register))
    # Um_circ.name = "U^m"
    # controlled_Um_gate = QControlBox(CircBox(Um_circ), 1)

    qpe_circ.U1(omega/np.pi, measurement_register[0])
    qpe_circ.H(measurement_register[0])
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [4]:
"""
runs phase estimation given the parameters, and then returns which outcome was most frequent
out of n_shots
"""
def get_next_digit(m: int, omega: float, state_prep_circuit: Circuit, unitary_circuit: Circuit,
                   backend=AerStateBackend(), n_shots=1000):
    qpe_circ = build_phase_estimation_circuit(m, omega, state_prep_circuit, unitary_circuit)
    compiled_circ = backend.get_compiled_circuit(qpe_circ)
    result = backend.run_circuit(compiled_circ, n_shots)
    sorted_shots = result.get_counts().most_common()
    print(sorted_shots)
    return sorted_shots[0][0][0] 

In [5]:
"""
iterates on qpe, updating the algorithm adaptively
"""
def iterated_qpe(state_prep_circuit: Circuit, unitary_circuit: Circuit, precision: int):
    digits = []
    for i in range(precision-1,-1,-1):
        omega = 0
        for j in range(len(digits)):
            omega = omega - np.pi*digits[j]/2**(j+1)
        x = get_next_digit(2**i, omega, state_prep_circuit, unitary_circuit)
        digits.insert(0,x)
    bitstring = "".join([str(bit) for bit in digits])
    print(bitstring)
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

In [7]:
### SIMPLE EXAMPLE
input_angle = 0.52536912 # set this to what you please
true_phase = input_angle/2

prep_circuit = Circuit(1).X(0)  # prepare the |1> eigenstate of U1
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops

estimated_phase = iterated_qpe(prep_circuit, unitary_circuit, precision=8)

print("Estimated Phase:", estimated_phase)
print("True Phase:", true_phase)

error = round(abs(input_angle - (2 * estimated_phase)), 3)
print("Error:",error)

[((1,), 860), ((0,), 140)]
[((1,), 959), ((0,), 41)]
[((0,), 992), ((1,), 8)]
[((0,), 995), ((1,), 5)]
[((0,), 1000)]
[((0,), 1000)]
[((1,), 1000)]
[((0,), 1000)]
01000011
Estimated Phase: 0.26171875
True Phase: 0.26268456
Error: 0.002
